#Chapter 6 Data loading, storage, file formats

In [2]:
import pandas as pd

## Read files
+ `.read_csv(path, sep= , header= , names=[list of column names], index_col=[list of index col names],`
    `skiprows=[rows to skip], na_values=[values considered as NA, or a dict], pase_dates=[list of columns to parse],'
    `date_parser=func to parse date, nrows=rows to be read, chunksize=#of chunks desired)` - and many other formats  
+ 

In [6]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)
tot = pd.Series([])
for pieces in chunker:
    tot=tot.add(pieces['key'].value_counts(), fill_value=0)

In [9]:
tot.order(ascending=False)

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
V    328
I    327
U    326
P    324
D    320
A    320
R    318
Y    314
G    308
S    308
N    306
W    305
T    304
B    302
Z    288
C    286
4    171
6    166
7    164
8    162
3    162
5    157
2    152
0    151
9    150
1    146
dtype: float64